### Data Kemdikbud scraper

Because somehow they make it very painful to just download the data.

In [1]:
import urllib
from bs4 import BeautifulSoup
import pandas as pd

Get list of kabupaten/kota

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/000000/"

# Mengirim permintaan HTTP ke halaman web dan mendapatkan kontennya
response = urlopen(url)

# Membuat objek BeautifulSoup dari konten halaman web
soup = BeautifulSoup(response, 'html.parser')

# Lakukan operasi lain pada objek BeautifulSoup
# ...



URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:992)>

In [4]:
master = urllib.urlopen("http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/000000/")
mastersoup = BeautifulSoup(master)

AttributeError: module 'urllib' has no attribute 'urlopen'

In [3]:
list_daerah=[]
for opt in mastersoup.find_all('option'):
    daerah={}
    sc_code = opt.get('value')
    if sc_code:
        daerah["code"]=sc_code
        daerah["name"]=''.join(opt.findAll(text=True))
        list_daerah.append(daerah)
list_daerah = list_daerah[:-8]
list_daerah

[{'code': '060000', 'name': u'Prop. Aceh'},
 {'code': '220000', 'name': u'Prop. Bali'},
 {'code': '290000', 'name': u'Prop. Bangka Belitung'},
 {'code': '280000', 'name': u'Prop. Banten'},
 {'code': '260000', 'name': u'Prop. Bengkulu'},
 {'code': '040000', 'name': u'Prop. D.I. Yogyakarta'},
 {'code': '010000', 'name': u'Prop. D.K.I. Jakarta'},
 {'code': '300000', 'name': u'Prop. Gorontalo'},
 {'code': '100000', 'name': u'Prop. Jambi'},
 {'code': '020000', 'name': u'Prop. Jawa Barat'},
 {'code': '030000', 'name': u'Prop. Jawa Tengah'},
 {'code': '050000', 'name': u'Prop. Jawa Timur'},
 {'code': '130000', 'name': u'Prop. Kalimantan Barat'},
 {'code': '150000', 'name': u'Prop. Kalimantan Selatan'},
 {'code': '140000', 'name': u'Prop. Kalimantan Tengah'},
 {'code': '160000', 'name': u'Prop. Kalimantan Timur'},
 {'code': '340000', 'name': u'Prop. Kalimantan Utara'},
 {'code': '310000', 'name': u'Prop. Kepulauan Riau'},
 {'code': '120000', 'name': u'Prop. Lampung'},
 {'code': '210000', 'name

In [5]:
dict_daerah={}
for i in list_daerah:
    dict_daerah[i["code"]]=i["name"][6:]
print(dict_daerah)

{'150000': u'Kalimantan Selatan', '200000': u'Sulawesi Tenggara', '280000': u'Banten', '140000': u'Kalimantan Tengah', '050000': u'Jawa Timur', '240000': u'Nusa Tenggara Timur', '220000': u'Bali', '130000': u'Kalimantan Barat', '030000': u'Jawa Tengah', '120000': u'Lampung', '300000': u'Gorontalo', '060000': u'Aceh', '020000': u'Jawa Barat', '190000': u'Sulawesi Selatan', '010000': u'D.K.I. Jakarta', '330000': u'Sulawesi Barat', '040000': u'D.I. Yogyakarta', '210000': u'Maluku', '160000': u'Kalimantan Timur', '290000': u'Bangka Belitung', '340000': u'Kalimantan Utara', '170000': u'Sulawesi Utara', '260000': u'Bengkulu', '250000': u'Papua', '310000': u'Kepulauan Riau', '180000': u'Sulawesi Tengah', '230000': u'Nusa Tenggara Barat', '100000': u'Jambi', '270000': u'Maluku Utara', '320000': u'Papua Barat', '090000': u'Riau'}


In [6]:
codelist = [item['code'] for item in list_daerah]
codelist

['060000',
 '220000',
 '290000',
 '280000',
 '260000',
 '040000',
 '010000',
 '300000',
 '100000',
 '020000',
 '030000',
 '050000',
 '130000',
 '150000',
 '140000',
 '160000',
 '340000',
 '310000',
 '120000',
 '210000',
 '270000',
 '230000',
 '240000',
 '250000',
 '320000',
 '090000',
 '330000',
 '190000',
 '180000',
 '200000',
 '170000']

Scrape marker in the response

In [13]:
#test: jakarta, SD
URL = "http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/"
f = urllib.urlopen("http://sekolah.data.kemdikbud.go.id/index.php/cpetasebaran/index/020000/SD") 
soup = BeautifulSoup(f)
#sc = soup.find_all('script')
#sc

In [24]:
sc = soup.title.string
if not sc == "Database Error":
    print("")

In [6]:
sc_c = sc.strip(' \t\n\r').split(";")
sc_arr = []
for s in sc_c:
    sc_arr.append(s.strip(' \t\n\r'))

In [7]:
sc_name=[]
sc_latlong=[]
for line in sc_arr:
    school={}
    if "NPSN" in line:
        #url
        school["url"] = line.split("=")[7][25:61]
        #NPSN
        school["npsn"]= line.split("=")[5][36:44]
        #nama
        sc_str = line.split("=")[8]
        school["name"] = sc_str[9:sc_str.find("</a")] 
        sc_name.append(school)   
    if "latLng" in line:
        latlng = line.split("ng(")[1][0:20].split(",")
        school["lat"] = latlng[0]
        school["long"] = latlng[1]
        sc_latlong.append(school)
#merge two dict list
for n, l in zip(sc_name,sc_latlong):
    n.update(l)

load into pandas dataframe

In [8]:
d = pd.DataFrame.from_dict(sc_name)

In [9]:
d.head()

,lat,long,name,npsn,url
0,.9590000,112.9798000,SD NEGERI 29 TANJUNG JATI,30103050,A0EA3E8E-13B5-E111-A5F1-43EF2009C94F
1,1.3384000,113.343500,SD NEGERI 05 KENEPAI KOMPLEK,30103216,C0EC0D09-30F5-E011-BB31-77ED21DFD3A0
2,.2571000,110.0809000,SD NEGERI 54 SENDAUN,30104473,C0024309-30F5-E011-887A-FB1D0F86FC3B
3,-.0411000,110.370300,SD NEGERI 15 MELOBOH,30102162,709FED08-30F5-E011-8423-AFA413B369AC
4,.6046000,112.6671000,SD NEGERI 19 SIMPANG 4 NANGA SURUK,30102957,20C90509-30F5-E011-BFD5-CD46218A8A79


In [10]:
d['level'] = "SD"
#d_smp['level'] = "SMP"
#d_sma['level'] = "SMA"
#d_smk['level'] = "SMK"
#d_all = pd.concat([d,d_smp,d_sma,d_smk])
d.head()

,lat,long,name,npsn,url,level
0,.9590000,112.9798000,SD NEGERI 29 TANJUNG JATI,30103050,A0EA3E8E-13B5-E111-A5F1-43EF2009C94F,SD
1,1.3384000,113.343500,SD NEGERI 05 KENEPAI KOMPLEK,30103216,C0EC0D09-30F5-E011-BB31-77ED21DFD3A0,SD
2,.2571000,110.0809000,SD NEGERI 54 SENDAUN,30104473,C0024309-30F5-E011-887A-FB1D0F86FC3B,SD
3,-.0411000,110.370300,SD NEGERI 15 MELOBOH,30102162,709FED08-30F5-E011-8423-AFA413B369AC,SD
4,.6046000,112.6671000,SD NEGERI 19 SIMPANG 4 NANGA SURUK,30102957,20C90509-30F5-E011-BFD5-CD46218A8A79,SD


In [52]:
#assuming separate dataframe are made for each level
d_all = pd.concat([d,d_smp,d_sma,d_smk])

In [11]:
d.to_csv("kalbar.csv")